In [1]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from scipy.sparse import csr_matrix
import scipy.sparse as sp
from collections import defaultdict
from torch import nn
import torch.nn.functional as F
from torchvision import models
import time
from tqdm import tqdm

In [2]:
path='C:/Users/Zhouziyue/workspace/tensorflow2.5_torch2.1/AAAI_feedback/ml-100k'

In [3]:
ALL_TRAIN = False

class RecsysData(Dataset):

    def __init__(self, path="../data/recsys2"):
        # train or test
        self.device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

        print(f'loading [{path}]')
        self.mode_dict = {'train': 0, "test": 1}
        self.mode = self.mode_dict['train']
        train_file = path + '/train_imp.csv'
        test_file = path + '/test_imp.csv'
        self.path = path
        self.trainSize = 0
        self.testSize = 0

        df = pd.read_csv(train_file)
        self.trainUniqueUsers = pd.unique(df["users"])
        self.trainUser = df["users"].to_numpy()
        self.trainItem = df["pos_item"].to_numpy()
        self.trainSize = len(df)

        #print(np.min(self.trainUser), np.min(self.trainItem))

        df_test = pd.read_csv(test_file)
        self.testUniqueUsers = pd.unique(df_test["users"])
        self.testUser = df_test["users"].to_numpy()
        self.testItem = df_test["pos_item"].to_numpy()
        
        self.m_item = 1523
        self.n_user = 943
        print(self.m_item,self.n_user)
        
        self.testSize = len(df)

        if ALL_TRAIN:
            self.trainUser = np.concatenate((self.trainUser, self.testUser), axis=0)
            self.trainItem = np.concatenate((self.trainItem, self.testItem), axis=0)
            self.trainSize = self.trainSize + self.testSize
        
        self.Graph = None
        print(f"{self.trainSize} interactions for training")
        print(f"{self.testSize} interactions for testing")
        print(f"Sparsity : {(self.trainSize + self.testSize) / self.n_user / self.m_item}")

        # (users,items), bipartite graph
        self.UserItemNet = csr_matrix((np.ones(len(self.trainUser)), 
                                        (self.trainUser, self.trainItem)),
                                        shape=(self.n_user, self.m_item))
        

        # pre-calculate
        self.allPos = self.getUserPosItems(list(range(self.n_user)))
        self.testDict = self.__build_test()
        print(f"Ready to go")

    def _convert_sp_mat_to_sp_tensor(self, X):
        coo = X.tocoo().astype(np.float32)
        row = torch.Tensor(coo.row).long()
        col = torch.Tensor(coo.col).long()
        index = torch.stack([row, col])
        data = torch.FloatTensor(coo.data)
        return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))
        
    def saveRatingMatrix(self):
        test_ratings = csr_matrix((np.ones(len(self.testUser)), 
                                        (self.testUser, self.testItem)),
                                        shape=(self.n_user, self.m_item))
        sp.save_npz(self.path + '/train_mat.npz', self.UserItemNet)
        sp.save_npz(self.path + '/test_mat.npz', test_ratings)


    def getSparseGraph(self):
        print("loading adjacency matrix")
        if self.Graph is None:
            try:
                pre_adj_mat = sp.load_npz(self.path + '/s_pre_adj_mat.npz')
                print("successfully loaded...")
                norm_adj = pre_adj_mat
            except :
                print("generating adjacency matrix")
                s = time.time()
                adj_mat = sp.dok_matrix((self.n_user + self.m_item, self.n_user + self.m_item), dtype=np.float32)
                adj_mat = adj_mat.tolil()
                R = self.UserItemNet.tolil()
                adj_mat[:self.n_user, self.n_user:] = R
                adj_mat[self.n_user:, :self.n_user] = R.T
                adj_mat = adj_mat.todok()
                
                rowsum = np.array(adj_mat.sum(axis=1))
                d_inv = np.power(rowsum, -0.5).flatten()
                d_inv[np.isinf(d_inv)] = 0.
                d_mat = sp.diags(d_inv)
                
                norm_adj = d_mat.dot(adj_mat)
                norm_adj = norm_adj.dot(d_mat)
                norm_adj = norm_adj.tocsr()
                end = time.time()
                print(f"costing {end-s}s, saved norm_mat...")
                sp.save_npz(self.path + '/s_pre_adj_mat.npz', norm_adj)
                

            self.Graph = self._convert_sp_mat_to_sp_tensor(norm_adj)
            self.Graph = self.Graph.coalesce().to(self.device)

        return self.Graph


    def __build_test(self):
        test_data = defaultdict(set)
        for user, item in zip(self.testUser, self.testItem):
            test_data[user].add(item)

        return test_data

    def getUserItemFeedback(self, users, items):
        return np.array(self.UserItemNet[users, items]).astype('uint8').reshape((-1,))

    def getUserPosItems(self, users):
        posItems = []
        for user in users:
            posItems.append(self.UserItemNet[user].nonzero()[1])
        return posItems

In [4]:
def set_seed(seed):
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.manual_seed(seed)

def UniformSample(dataset:RecsysData):
    users = np.random.randint(0, dataset.n_user, dataset.trainSize)
    allPos = dataset.allPos
    S = []
    for user in users:
        posForUser = allPos[user]
        if len(posForUser) == 0:
            continue

        positem = posForUser[np.random.randint(0, len(posForUser))]
        negitem = np.random.randint(0, dataset.m_item)
        while negitem in posForUser:
            negitem = np.random.randint(0, dataset.m_item)

        S.append([user, positem, negitem])

    return np.array(S)

def shuffle(*arrays, **kwargs):

    require_indices = kwargs.get('indices', False)

    if len(set(len(x) for x in arrays)) != 1:
        raise ValueError('All inputs to shuffle must have the same length.')

    shuffle_indices = np.arange(len(arrays[0]))
    np.random.shuffle(shuffle_indices)

    if len(arrays) == 1:
        result = arrays[0][shuffle_indices]
    else:
        result = tuple(x[shuffle_indices] for x in arrays)

    if require_indices:
        return result, shuffle_indices
    else:
        return result

def generate_batches(tensors, batch_size):
    for i in range(0, len(tensors), batch_size):
        yield tensors[i:i + batch_size]

def my_generate_batches(tensors_list, batch_size):
    for i in range(0, len(tensors_list[0]), batch_size):
        yield tensors_list[0][i:i + batch_size], tensors_list[1][i:i + batch_size]
        
def minibatch(tensors, batch_size):
    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i:i + batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i:i + batch_size] for x in tensors)

def cust_mul(s, d, dim):
    i = s._indices()
    v = s._values()
    dv = d[i[dim,:]]
    return torch.sparse.FloatTensor(i, v * dv, s.size())


def calc_recall(ratings, test_data, k, users):
    num = ratings.sum(1)
    den = np.array([len(test_data[u]) for u in users]).astype('float')
    num[den == 0.] = 0.
    den[den == 0.] = 1.
    recall = np.sum(num/den)
    return recall

def calc_ndcg(ratings, test_data, k, users):
    test_matrix = np.zeros((len(users), k))
    for i,user in enumerate(users):
        length = k if k <= len(test_data[user]) else len(test_data[user])
        test_matrix[i, :length] = 1

    idcg = np.sum(test_matrix * 1./np.log2(np.arange(2, k + 2)), axis=1)
    idcg[idcg == 0.] = 1.
    dcg = ratings*(1./np.log2(np.arange(2, k + 2)))
    dcg = np.sum(dcg, axis=1)
    ndcg = np.sum(dcg/idcg)
    return ndcg

def calc_ncrr(ratings, test_data, k, users):
    fractions = [1.0/n for n in range(1,k+1)]
    fractions = np.array(fractions)
    crr = ratings.dot(fractions.T)

    accum = np.cumsum(fractions)
    icrr = np.array([accum[min(len(test_data[u])-1, k-1)] for u in users])
    icrr[icrr == 0.] = 1.

    ncrr = np.sum(crr/icrr)
    return ncrr

def calc_hm(a, b, c):
    return 3/(1/a + 1/b + 1/c)


In [5]:
class IMP_GCN(nn.Module):
    def __init__(self,  
                 dataset: RecsysData,
                 latent_dim=200, 
                 n_layers=6, 
                 keep_prob=0.9,
                 groups=4,
                 device=torch.device('cuda'),
                 dropout_bool=False,
                 l2_w=1e-4,
                 single=False):
        super(IMP_GCN, self).__init__()
        self.latent_dim = latent_dim
        self.n_layers = n_layers
        self.dropout_bool = dropout_bool
        self.keep_prob = keep_prob
        self.Graph = dataset.getSparseGraph()
        self.num_users  = dataset.n_user
        self.num_items  = dataset.m_item
        self.groups = groups
        self.device = device
        self.l2_w = l2_w
        self.single = single
        self.__init_weight()

    def __init_weight(self):
        self.embedding_user = torch.nn.Embedding(self.num_users, self.latent_dim)
        self.embedding_item = torch.nn.Embedding(self.num_items, self.latent_dim)
        self.fc = torch.nn.Linear(self.latent_dim, self.latent_dim)
        self.leaky = torch.nn.LeakyReLU()
        self.dropout = torch.nn.Dropout(p=0.4)
        self.fc_g = torch.nn.Linear(self.latent_dim, self.groups)
        self.f = nn.Sigmoid()

        #nn.init.normal_(self.embedding_user.weight, std=0.1)
        #nn.init.normal_(self.embedding_item.weight, std=0.1)
        nn.init.xavier_uniform_(self.embedding_user.weight, gain=1)
        nn.init.xavier_uniform_(self.embedding_item.weight, gain=1)
        #nn.init.xavier_uniform_(self.fc.weight, gain=1)
        #nn.init.xavier_uniform_(self.fc_g.weight, gain=1)

    def __dropout_x(self, x, keep_prob):
        size = x.size()
        index = x.indices().t()
        values = x.values()
        random_index = torch.rand(len(values)) + keep_prob
        random_index = random_index.int().bool()
        index = index[random_index]
        values = values[random_index]/keep_prob
        g = torch.sparse.FloatTensor(index.t(), values, size)
        return g
    
    def __dropout(self, keep_prob):
        graph = self.__dropout_x(self.Graph, keep_prob)
        return graph
    

    def computer(self):    
        users_emb = self.embedding_user.weight
        items_emb = self.embedding_item.weight
        all_emb = torch.cat([users_emb, items_emb])
        
        if self.dropout_bool and self.training:
            g_droped = self.__dropout(self.keep_prob)     
        else:
            g_droped = self.Graph
        
        # Compute ego + side embeddings
        ego_embed = all_emb
        side_embed = torch.sparse.mm(g_droped, all_emb)
        
        temp = self.dropout(self.leaky(self.fc(ego_embed + side_embed)))
        group_scores = self.dropout(self.fc_g(temp))
        #group_scores = self.fc_g(temp)

        a_top, a_top_idx = torch.topk(group_scores, k=1, sorted=False)
        one_hot_emb = torch.eq(group_scores,a_top).float()

        u_one_hot, i_one_hot = torch.split(one_hot_emb, [self.num_users, self.num_items])
        i_one_hot = torch.ones(i_one_hot.shape).to(self.device)
        one_hot_emb = torch.cat([u_one_hot, i_one_hot]).t()

        # Create Subgraphs
        subgraph_list = []
        for g in range(self.groups):
            temp = cust_mul(g_droped, one_hot_emb[g], 1)
            temp = cust_mul(temp, one_hot_emb[g], 0)
            subgraph_list.append(temp)

        all_emb_list = [[None for _ in range(self.groups)] for _ in range(self.n_layers)]
        for g in range(0,self.groups):
            all_emb_list[0][g] = ego_embed
        
        for k in range(1,self.n_layers):
            for g in range(self.groups):
                all_emb_list[k][g] = torch.sparse.mm(subgraph_list[g], all_emb_list[k-1][g])

        
        all_emb_list = [torch.sum(torch.stack(x),0) for x in all_emb_list]
        
        if self.single:
            all_emb = all_emb_list[-1]
        else:
            weights = [0.2, 0.2, 0.2, 0.2, 0.2]
            all_emb_list = [x * w for x,w in zip(all_emb_list,weights)]
            all_emb = torch.sum(torch.stack(all_emb_list),0)
            #all_emb = torch.mean(torch.stack(all_emb_list),0)
            #all_emb = all_emb_list[-1]

        users, items = torch.split(all_emb, [self.num_users, self.num_items])
        return users, items
    
    def getUsersRating(self, users):
        all_users, all_items = self.computer()
        users_emb = all_users[users.long()]
        items_emb = all_items
        rating = self.f(torch.matmul(users_emb, items_emb.t()))
        return rating
    
    def my_getUsersRating(self, users,targets):
        #items = torch.arange(self.item_num).to(users.device)
        all_users, all_items = self.computer()
        user_embeds = all_users[users].unsqueeze(1)#[b,1,h]
        tar_item_embeds = all_items[targets].unsqueeze(1)#[b,1,h]
        #neg_indx = torch.randint(low=1, high=self.num_items, size=(tar_item_embeds.shape[0], 1000))#.to(users.device)  
        #neg_indx = torch.arange(self.item_num).to(users.device).unsqueeze(0)
        neg_item_embeds = all_items.expand(tar_item_embeds.shape[0], -1, -1)
        test_item_embeds=torch.cat([tar_item_embeds,neg_item_embeds],dim=1)#[b,1+1000,h]
        #print('user_embeds',user_embeds.shape)
        #print('test_item_embeds',test_item_embeds.shape)
        
        return torch.matmul(user_embeds,neg_item_embeds.transpose(1,2)).squeeze(1),\
    torch.matmul(user_embeds,tar_item_embeds.transpose(1,2)).squeeze(1)
    
    
    def getEmbedding(self, users, pos_items, neg_items):
        all_users, all_items = self.computer()
        users_emb = all_users[users]
        pos_emb = all_items[pos_items]
        neg_emb = all_items[neg_items]
        users_emb_ego = self.embedding_user(users)
        pos_emb_ego = self.embedding_item(pos_items)
        neg_emb_ego = self.embedding_item(neg_items)
        return users_emb, pos_emb, neg_emb, users_emb_ego, pos_emb_ego, neg_emb_ego
    
    def bpr_loss(self, users, pos, neg):
        (users_emb, pos_emb, neg_emb, 
        userEmb0,  posEmb0, negEmb0) = self.getEmbedding(users.long(), pos.long(), neg.long())
        reg_loss = (1/2)*(userEmb0.norm(2).pow(2) + 
                         posEmb0.norm(2).pow(2)  +
                         negEmb0.norm(2).pow(2))/float(len(users))
        pos_scores = torch.mul(users_emb, pos_emb)
        pos_scores = torch.sum(pos_scores, dim=1)
        neg_scores = torch.mul(users_emb, neg_emb)
        neg_scores = torch.sum(neg_scores, dim=1)
        
        loss = torch.mean(F.softplus(neg_scores - pos_scores))
        
        return loss + self.l2_w * reg_loss
       
    def forward(self, users, items):
        all_users, all_items = self.computer()
        users_emb = all_users[users]
        items_emb = all_items[items]
        inner_pro = torch.mul(users_emb, items_emb)
        gamma     = torch.sum(inner_pro, dim=1)
        return gamma


        all_users, all_items = self.computer()
        users_emb = all_users[users]
        items_emb = all_items[items]
        inner_pro = torch.mul(users_emb, items_emb)
        gamma     = torch.sum(inner_pro, dim=1)
        return gamma


In [6]:
#from dataloader import RecsysData
#from model import LightGCN, IMP_GCN

dataset = RecsysData(path)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")



model1 = IMP_GCN(dataset, latent_dim=64, n_layers=3, groups=3, dropout_bool=True, l2_w=0.0002, single=True).to(device)
optim1 = torch.optim.Adam(model1.parameters(), lr=0.001)
args1 = {"dataset": dataset, 
        "model": model1,
        "optimiser": optim1,
        "filename": f"imp_gcn_s_d{300}_l{3}_reg{2}_lr{'001'}"}

loading [C:/Users/Zhouziyue/workspace/tensorflow2.5_torch2.1/AAAI_feedback/ml-100k]
1523 943
58196 interactions for training
58196 interactions for testing
Sparsity : 0.08104225836571649
Ready to go
loading adjacency matrix
successfully loaded...


C:\Users\Zhouziyue\AppData\Local\Temp\ipykernel_29636\430940866.py:64: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:607.)
  return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))


In [7]:
#import utils
SEED = 2021
set_seed(SEED)

#from dataloader import RecsysData, ALL_TRAIN
#from model_configs import dataset, device, args1, args2
#from model import LightGCN, IMP_GCN



EPOCHS = 50
TEST_INTERVAL = 1
BATCH_SIZE = 1024

TOP_K = 10

def run_train(dataset, model, optimiser):
    model.train()
    num_batches = dataset.trainSize // BATCH_SIZE + 1
    mean_batch_loss = 0

    S = UniformSample(dataset)
    users = torch.Tensor(S[:, 0]).long().to(device)
    posItems = torch.Tensor(S[:, 1]).long().to(device)
    negItems = torch.Tensor(S[:, 2]).long().to(device)
    users, posItems, negItems = shuffle(users, posItems, negItems)

    for i,(b_users, b_pos, b_neg) in enumerate(minibatch((users,posItems,negItems), BATCH_SIZE)):
        #bpr_loss, reg_loss = model.bpr_loss(b_users, b_pos, b_neg)
        #loss = bpr_loss + reg_loss*L2_W
        loss = model.bpr_loss(b_users, b_pos, b_neg)
        mean_batch_loss += loss.cpu().item()

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        print(f"  Batch {i+1}/{num_batches}: loss = {loss.cpu().item():.8f}" + " " * 20, end='\r')
    

    return f"Final train loss: {mean_batch_loss/num_batches:.7f}"

def run_test(dataset, model):
    model.eval()
    print("-"*40)
    print("TEST RESULTS")
    print("-"*40)
    
    test_data = dataset.testDict
    test_users = list(test_data.keys())
    test_batch_size = 100
    cur_idx = 0
    
    t_recall = 0
    t_ndcg = 0
    t_ncrr = 0
    num_batches = len(test_users) // test_batch_size + 1
    for batch_users in tqdm(generate_batches(test_users, test_batch_size), total=num_batches):
        with torch.no_grad():
            allPos = dataset.getUserPosItems(batch_users)
            batch_users_gpu = torch.Tensor(batch_users).long().to(device)
            all_ratings = model.getUsersRating(batch_users_gpu)
            exclude_index = []
            exclude_items = []

            for range_i, items in enumerate(allPos):
                exclude_index.extend([range_i] * len(items))
                exclude_items.extend(items)
            all_ratings[exclude_index, exclude_items] = -(1<<10)

            _, top_k_ratings = torch.topk(all_ratings, TOP_K, dim=1)
            top_k_ratings = top_k_ratings.cpu().numpy()
            del all_ratings
        
        r = []
        for i,u in enumerate(batch_users):
            pred = list(map(lambda x: x in test_data[u], top_k_ratings[i]))
            pred = np.array(pred).astype('float')
            r.append(pred)
        r = np.array(r).astype('float')

        t_recall +=calc_recall(r, test_data, TOP_K, batch_users)
        t_ndcg += calc_ndcg(r, test_data, TOP_K, batch_users)
        t_ncrr += calc_ncrr(r, test_data, TOP_K, batch_users)
    
    t_recall /= len(test_users)
    t_ndcg /= len(test_users)
    t_ncrr /= len(test_users)
    hm = calc_hm(t_recall, t_ndcg, t_ncrr)

    print(f"Recall: {t_recall}\n" + f"NDCG: {t_ndcg}\n" + f"NCRR: {t_ncrr}\n" + f"HM: {hm}")
    print("-"*40)
    return hm
    return "hm_" + f"{hm:.5f}"[2:]

def hr(rank, k):
    """Hit Rate.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: hit rate.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1
    return res / len(rank)


def mrr(rank, k):
    """Mean Reciprocal Rank.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: mrr.
    """
    mrr = 0.0
    for r in rank:
        if r < k:
            mrr += 1 / (r + 1)
    return mrr / len(rank)


def ndcg(rank, k):
    """Normalized Discounted Cumulative Gain.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: ndcg.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1 / np.log2(r + 2)
    return res / len(rank)

def my_run_test(dataset, model):
    model.eval()
    print("-"*40)
    print("TEST RESULTS")
    print("-"*40)
    
    test_data = [dataset.testUser,dataset.testItem]
    test_users = list(test_data[0])
    test_batch_size = 1024
    cur_idx = 0
    
    hr_b=[]
    mrr_b=[]
    ndcg_b=[]
    acc_b=[]
    num_batches = len(test_data) // test_batch_size + 1
    for batch_users,batch_target in tqdm(my_generate_batches(test_data, test_batch_size), total=num_batches):
        with torch.no_grad():
            allPos = dataset.getUserPosItems(batch_users)
            batch_users_gpu = torch.Tensor(batch_users).long().to(device)
            all_rating,pos_rating = model.my_getUsersRating(batch_users,batch_target)
            print('rating shape:',all_rating.shape,pos_rating.shape)
            #rating = rating.cpu()
            #all_rating += mask[batch_users]
            rating=-torch.cat([pos_rating,all_rating],dim=1)
            rank = rating.argsort().argsort()[:, 0]
            rank=rank.cpu()
            res_1 = hr(rank, TOP_K)
            res_2 = ndcg(rank, TOP_K)
            res_3 = mrr(rank, TOP_K)
            res_4 = hr(rank, 1)
            
            hr_b.append(res_1)
            mrr_b.append(res_3)
            ndcg_b.append(res_2)
            acc_b.append(res_4)
            
    f_hr=np.mean(hr_b)
    f_mrr=np.mean(mrr_b)
    f_ndcg=np.mean(ndcg_b)
    f_acc=np.mean(acc_b)
    print(f"hr: {f_hr}\n" + f"mrr: {f_mrr}\n" + f"ndcg: {f_ndcg}\n" + f"ac: {f_acc}")
    print("-"*40)
    return f_hr, f_mrr, f_ndcg, f_acc   


def main(args):
    dataset = args['dataset']
    model = args['model']
    optimiser = args['optimiser']
    filename = args['filename']
    best_epoch, best_recall, best_ndcg, best_hr = 0, 0, 0, 0
    early_stop=False
    for epoch in range(1,EPOCHS+1):
        start = time.time()

        train_info = run_train(dataset, model, optimiser)

        total_time = time.time() - start
        output_info = f'[Time taken: {total_time:.1f}s]'
        print(f'Epoch[{epoch}/{EPOCHS}]   {train_info}  {output_info}')

        if epoch % TEST_INTERVAL == 0:
            if ALL_TRAIN:
                generate_submission(model, dataset, f"submissions/final_{epoch}_2_sub.txt")
            else:
                
                '''hm = run_test(dataset, model)
                str_hm = "hm_" + f"{hm:.5f}"[2:]
                temp = f"{filename}_e{epoch}_{str_hm}"'''
                f_hr, f_mrr, f_ndcg, f_acc=my_run_test(dataset, model)
            if f_hr > best_hr:
                best_hr, best_mrr, best_ndcg,best_acc, best_epoch = f_hr, f_mrr, f_ndcg, f_acc, epoch
                early_stop_count = 0

            else:
                early_stop_count += 1
                if early_stop_count == 3:
                    early_stop = True
        
        if early_stop:
            print('##########################################')
            print('Early stop is triggered at {} epochs.'.format(epoch))
            break
main(args1)

Epoch[1/50]   Final train loss: 0.6776540  [Time taken: 1.4s]
----------------------------------------
TEST RESULTS
----------------------------------------


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

rating shape: torch.Size([1024, 1523]) torch.Size([1024, 1])


2it [00:00,  6.88it/s]                                                                                                 

rating shape: torch.Size([1024, 1523]) torch.Size([1024, 1])


...

rating shape: torch.Size([1024, 1523]) torch.Size([1024, 1])
rating shape: torch.Size([1024, 1523]) torch.Size([1024, 1])


14it [00:01,  8.58it/s]

rating shape: torch.Size([1024, 1523]) torch.Size([1024, 1])
rating shape: torch.Size([1024, 1523]) torch.Size([1024, 1])


16it [00:01,  8.48it/s]

rating shape: torch.Size([1024, 1523]) torch.Size([1024, 1])
rating shape: torch.Size([738, 1523]) torch.Size([738, 1])
hr: 0.04792946916285569
mrr: 0.013846385292708874
ndcg: 0.021685262485288972
ac: 0.0036956869812838754
----------------------------------------
##########################################
Early stop is triggered at 6 epochs.
